In [1]:
import numpy as np
import pandas as pd

# Importing data

### Becelona Data

In [2]:
#barcelona_calendar = pd.read_csv("barcelona_calendar.csv")
barcelona_listings = pd.read_csv("barcelona_listings.csv")
#barcelona_reviews = pd.read_csv("barcelona_reviews.csv")

### Berlin Data

In [3]:
#berlin_calendar = pd.read_csv("berlin_calendar.csv")
berlin_listings = pd.read_csv("berlin_listings_filtered.csv")
#berlin_reviews = pd.read_csv("berlin_reviews.csv")

### Madrid Data

In [4]:
#madrid_calendar = pd.read_csv("madrid_calendar.csv")
madrid_listings = pd.read_csv("madrid_listings_filtered.csv")
#madrid_reviews = pd.read_csv("madrid_reviews.csv")

In [7]:
berlin_listings['amenities'][0]

'{TV,"Cable TV",Wifi,Kitchen,"Free street parking",Heating,"Family/kid friendly",Washer,"Smoke detector","Carbon monoxide detector","First aid kit","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","Self check-in",Lockbox,"Private living room","Private entrance","Baby bath","High chair","Pack ’n Play/travel crib","Room-darkening shades","Hot water","Bed linens","Extra pillows and blankets","Ethernet connection","Coffee maker",Refrigerator,Dishwasher,"Dishes and silverware","Cooking basics",Stove,"Single level home","Patio or balcony","Garden or backyard","Long term stays allowed","Paid parking on premises"}'

# Data management
## amenities
### amenities: AMENITY; AMENITY_ID

In [11]:
all_amenities = pd.concat([berlin_listings['amenities'], madrid_listings['amenities'], barcelona_listings['amenities']]) #concatenate all amenities
amenities = all_amenities.str.replace('{','').str.replace('}','').str.replace('"', '') #remove '{' , '}' and '"' 
amenities = amenities.str.split(',', expand = True) #seperate all amenities
a = amenities.sort_index().values #change into a np array
a = a.ravel() #ravel into a 1D array
print(a)
a = a[a != None] #removes all None
a = np.unique(a) #same as drop_duplicates
a = a[1:] #the first one was an empty string
amenities = pd.Series(a) #to pd
amenities = amenities.sort_values() # those are the amenities sorted
#note: the toilet amenity has a space at the beginning

['TV' 'Cable TV' 'Wifi' ..., None None None]


In [9]:
amenities = pd.DataFrame(amenities).rename(index = str, columns = {0 : 'AMENITY'}) #gives amenities column a name
amenities['AMENITY_ID'] = range(amenities['AMENITY'].count()) #create indexes for each amenity
amenities= amenities.set_index('AMENITY_ID') #defines the created indexes as the index

In [10]:
amenities.head(5)

,AMENITY
AMENITY_ID,
0,toilet
1,24-hour check-in
2,Accessible-height bed
3,Accessible-height toilet
4,Air conditioning


In [8]:
amenities.to_csv('amenities.csv')

### amenities_AND_listings: AMENITY_ID; LISTING_ID

In [9]:
listings_and_amenities = pd.concat([berlin_listings[['id','amenities']], madrid_listings[['id','amenities']], barcelona_listings[['id','amenities']]]) #concatenate all amenities with listings_id

In [10]:
am = listings_and_amenities['amenities'].str.replace('{','').str.replace('}','').str.replace('"','') 
am = am.map(lambda x: np.array(x.split(','))) #list of list of amenities
d = amenities.to_dict()['AMENITY']
inv_d = dict([[v,k] for k,v in d.items()]) #setup the mapping from amenities to amenity_id
am = am.map(lambda am_list : [inv_d[x] for x in am_list if len(am_list)>1]) #all lists of all amenities_id
am.head()

0    [159, 27, 176, 94, 69, 83, 58, 166, 148, 28, 6...
1    [176, 94, 149, 26, 83, 166, 54, 140, 1, 77, 17...
2    [159, 90, 176, 94, 117, 149, 26, 83, 166, 48, ...
3    [159, 90, 176, 94, 69, 26, 83, 58, 166, 63, 60...
4    [159, 176, 94, 69, 26, 83, 58, 148, 63, 60, 54...
Name: amenities, dtype: object

In [11]:
listings_and_amenities.drop(['amenities'], axis = 1)
listings_and_amenities['amenities'] = am
listings_and_amenities.head()

,id,amenities
0,2137277,"[159, 27, 176, 94, 69, 83, 58, 166, 148, 28, 6..."
1,2138961,"[176, 94, 149, 26, 83, 166, 54, 140, 1, 77, 17..."
2,2140695,"[159, 90, 176, 94, 117, 149, 26, 83, 166, 48, ..."
3,2147642,"[159, 90, 176, 94, 69, 26, 83, 58, 166, 63, 60..."
4,2147777,"[159, 176, 94, 69, 26, 83, 58, 148, 63, 60, 54..."


In [53]:
pd.DataFrame(listings_and_amenities.amenities.tolist(), index=listings_and_amenities.id).stack().reset_index(level=1, drop=True).reset_index(name='amenities').head()

,id,amenities
0,2137277,159.0
1,2137277,27.0
2,2137277,176.0
3,2137277,94.0
4,2137277,69.0


In [54]:
#am_lis_np = listings_and_amenities.values
#temp = [np.column_stack((np.repeat(x[0], len(x[1])), np.array(x[1]))) for x in am_lis_np]
#listings_and_amenities_np = np.array([])
#for x in temp:
#    listings_and_amenities_np = np.append(listings_and_amenities_np, x)

In [57]:
#listings_and_amenities_np = listings_and_amenities_np.reshape(int(listings_and_amenities_np.shape[0]/2), 2)

In [58]:
#listings_and_amenities_np = listings_and_amenities_np.astype('int64')

In [58]:
#listings_and_amenities = pd.DataFrame({'LISTING_ID': listings_and_amenities_np[:,0], 'AMENITY_ID': listings_and_amenities_np[:,1]})
#listings_and_amenities.head()

,LISTING_ID,AMENITY_ID
0,2137277,159
1,2137277,27
2,2137277,176
3,2137277,94
4,2137277,69


In [59]:
listings_and_amenities.to_csv('amenities_AND_listings.csv', index = False)

## Hosts

### host_response_time: HOST_RESPONSE_TIME_ID; HOST_RESPONSE_TIME

In [235]:
# Those are all the hosts of all the different listings
listingsB_hosts = berlin_listings.iloc[:, 13:24]
listingsM_hosts = madrid_listings.iloc[:, 13:24]
listingsBa_hosts = barcelona_listings.iloc[:, 13:24]
# We concatenate hosts from all cities
listings_hosts = pd.concat([listingsB_hosts, listingsM_hosts, listingsBa_hosts])
# we remove host duplicates since a host can have several listings.
hosts = listings_hosts.drop_duplicates(subset = 'host_id', keep = 'first')

In [236]:
host_resp_time = pd.DataFrame(hosts['host_response_time'].drop_duplicates(keep = 'first')) #filters the host_response_time
host_resp_time['host_response_time_id'] = range(host_resp_time['host_response_time'].count()+1) #creates indexes
host_resp_time = host_resp_time.set_index('host_response_time_id') #set new indexes
host_resp_time.head()

,host_response_time
host_response_time_id,
0,within an hour
1,NaN
2,within a day
3,within a few hours
4,a few days or more


In [237]:
host_resp_time.to_csv('host_response_time.csv')

### host_verifications: HOST_VERIFICATION; HOST_VERIFICATION_ID

In [238]:
hosts_verifications = hosts['host_verifications'].str.replace('[','').str.replace(']','').str.replace("'",'').str.split(',',expand = True)
hosts_verifications = hosts_verifications.values.flatten()
hosts_verifications = hosts_verifications[hosts_verifications != None]
hosts_verifications = np.unique(hosts_verifications)[1:].astype('str')
hosts_verifications

array([' facebook', ' google', ' government_id', ' identity_manual',
       ' jumio', ' kba', ' manual_offline', ' manual_online',
       ' offline_government_id', ' phone', ' photographer', ' reviews',
       ' selfie', ' sent_id', ' sesame', ' sesame_offline', ' weibo',
       ' work_email', ' zhima_selfie', 'email', 'facebook',
       'identity_manual', 'jumio', 'phone', 'reviews'], dtype='<U22')

In [239]:
verifications = pd.DataFrame({'HOST_VERIFICATION' : hosts_verifications, 'HOST_VERIFICATION_ID' : np.arange(hosts_verifications.shape[0])})
verifications = verifications.set_index('HOST_VERIFICATION_ID')
verifications.head()

,HOST_VERIFICATION
HOST_VERIFICATION_ID,
0,facebook
1,google
2,government_id
3,identity_manual
4,jumio


In [252]:
verifications.to_csv('host_verifications.csv', encoding = 'utf-8')

## hosts

In [241]:
hosts = hosts.set_index('host_id') #setting up index
hosts = hosts.drop(['host_verifications'], axis = 1)
temp = hosts['host_response_rate'].str.replace('%','').values #create new pd.Series of new host_response_rates
hosts = hosts.drop(['host_response_rate'], axis = 1) #remove previous column
hosts['host_response_rate'] = temp #add new column
del(temp)
hosts.head()

,host_url,host_name,host_since,host_about,host_response_time,host_thumbnail_url,host_picture_url,host_neighbourhood,host_response_rate
host_id,,,,,,,,,
10692949,https://www.airbnb.com/users/show/10692949,Suites & More,2013-12-18,Like hosts we like to offer what we would like...,within an hour,https://a0.muscache.com/im/users/10692949/prof...,https://a0.muscache.com/im/users/10692949/prof...,Kreuzberg,100
10914898,https://www.airbnb.com/users/show/10914898,Anita,2013-12-30,NaN,NaN,https://a0.muscache.com/im/pictures/37a4e6f0-6...,https://a0.muscache.com/im/pictures/37a4e6f0-6...,Friedrichshain,NaN
4314868,https://www.airbnb.com/users/show/4314868,Katharina,2012-12-03,Ich lebe in Berlin - im schönen Prenzlauer Ber...,within an hour,https://a0.muscache.com/im/users/4314868/profi...,https://a0.muscache.com/im/users/4314868/profi...,Prenzlauer Berg,90
9828152,https://www.airbnb.com/users/show/9828152,Katrin,2013-11-05,"Wir sind leidenschaftliche Berliner, lieben un...",NaN,https://a0.muscache.com/im/users/9828152/profi...,https://a0.muscache.com/im/users/9828152/profi...,Alt-Hohenschönhausen,NaN
10962834,https://www.airbnb.com/users/show/10962834,Katharina,2014-01-02,NaN,NaN,https://a0.muscache.com/im/users/10962834/prof...,https://a0.muscache.com/im/users/10962834/prof...,Moabit,NaN


### changing the host_response_time with thier ids

In [242]:
d = host_resp_time.to_dict()['host_response_time']
inv_d = dict([[v,k] for k,v in d.items()])
host_resp_time = hosts['host_response_time'].map(inv_d, na_action = 'ignore')

In [243]:
hosts = hosts.drop(['host_response_time'], axis = 1)
hosts['host_response_time_id'] = host_resp_time
hosts.head()

,host_url,host_name,host_since,host_about,host_thumbnail_url,host_picture_url,host_neighbourhood,host_response_rate,host_response_time_id
host_id,,,,,,,,,
10692949,https://www.airbnb.com/users/show/10692949,Suites & More,2013-12-18,Like hosts we like to offer what we would like...,https://a0.muscache.com/im/users/10692949/prof...,https://a0.muscache.com/im/users/10692949/prof...,Kreuzberg,100,0
10914898,https://www.airbnb.com/users/show/10914898,Anita,2013-12-30,NaN,https://a0.muscache.com/im/pictures/37a4e6f0-6...,https://a0.muscache.com/im/pictures/37a4e6f0-6...,Friedrichshain,NaN,1
4314868,https://www.airbnb.com/users/show/4314868,Katharina,2012-12-03,Ich lebe in Berlin - im schönen Prenzlauer Ber...,https://a0.muscache.com/im/users/4314868/profi...,https://a0.muscache.com/im/users/4314868/profi...,Prenzlauer Berg,90,0
9828152,https://www.airbnb.com/users/show/9828152,Katrin,2013-11-05,"Wir sind leidenschaftliche Berliner, lieben un...",https://a0.muscache.com/im/users/9828152/profi...,https://a0.muscache.com/im/users/9828152/profi...,Alt-Hohenschönhausen,NaN,1
10962834,https://www.airbnb.com/users/show/10962834,Katharina,2014-01-02,NaN,https://a0.muscache.com/im/users/10962834/prof...,https://a0.muscache.com/im/users/10962834/prof...,Moabit,NaN,1


### changing the host_neighborhood by their ids

In [244]:
di = pd.read_csv('neighbourhood.csv', encoding = 'utf-8')['NEIGH'].to_dict()
#creates a maping like before, need to reverse it:
inv_di = dict([[v,k] for k,v in di.items()])
hosts_neigh_id = hosts['host_neighbourhood'].map(inv_di)
hosts_neigh_id[:5]

host_id
10692949    154
10914898    155
4314868     156
9828152     157
10962834    158
Name: host_neighbourhood, dtype: int64

In [245]:
hosts = hosts.drop(['host_neighbourhood'], axis = 1)
hosts['neigh_id'] = hosts_neigh_id
hosts.head()

,host_url,host_name,host_since,host_about,host_thumbnail_url,host_picture_url,host_response_rate,host_response_time_id,neigh_id
host_id,,,,,,,,,
10692949,https://www.airbnb.com/users/show/10692949,Suites & More,2013-12-18,Like hosts we like to offer what we would like...,https://a0.muscache.com/im/users/10692949/prof...,https://a0.muscache.com/im/users/10692949/prof...,100,0,154
10914898,https://www.airbnb.com/users/show/10914898,Anita,2013-12-30,NaN,https://a0.muscache.com/im/pictures/37a4e6f0-6...,https://a0.muscache.com/im/pictures/37a4e6f0-6...,NaN,1,155
4314868,https://www.airbnb.com/users/show/4314868,Katharina,2012-12-03,Ich lebe in Berlin - im schönen Prenzlauer Ber...,https://a0.muscache.com/im/users/4314868/profi...,https://a0.muscache.com/im/users/4314868/profi...,90,0,156
9828152,https://www.airbnb.com/users/show/9828152,Katrin,2013-11-05,"Wir sind leidenschaftliche Berliner, lieben un...",https://a0.muscache.com/im/users/9828152/profi...,https://a0.muscache.com/im/users/9828152/profi...,NaN,1,157
10962834,https://www.airbnb.com/users/show/10962834,Katharina,2014-01-02,NaN,https://a0.muscache.com/im/users/10962834/prof...,https://a0.muscache.com/im/users/10962834/prof...,NaN,1,158


### changing host_about by removing \n from string

In [253]:
host_about = hosts['host_about'].str.replace('\n', '').str.replace(',','')
hosts = hosts.drop(['host_about'], axis = 1)
hosts['host_about'] = host_about
hosts = hosts.rename({'host_response_time' : 'host_response_time_id'})
hosts.head()

,host_url,host_name,host_since,host_thumbnail_url,host_picture_url,host_response_rate,host_response_time_id,neigh_id,host_about
host_id,,,,,,,,,
10692949,https://www.airbnb.com/users/show/10692949,Suites & More,2013-12-18,https://a0.muscache.com/im/users/10692949/prof...,https://a0.muscache.com/im/users/10692949/prof...,100,0,154,Like hosts we like to offer what we would like...
10914898,https://www.airbnb.com/users/show/10914898,Anita,2013-12-30,https://a0.muscache.com/im/pictures/37a4e6f0-6...,https://a0.muscache.com/im/pictures/37a4e6f0-6...,NaN,1,155,NaN
4314868,https://www.airbnb.com/users/show/4314868,Katharina,2012-12-03,https://a0.muscache.com/im/users/4314868/profi...,https://a0.muscache.com/im/users/4314868/profi...,90,0,156,Ich lebe in Berlin - im schönen Prenzlauer Ber...
9828152,https://www.airbnb.com/users/show/9828152,Katrin,2013-11-05,https://a0.muscache.com/im/users/9828152/profi...,https://a0.muscache.com/im/users/9828152/profi...,NaN,1,157,Wir sind leidenschaftliche Berliner lieben uns...
10962834,https://www.airbnb.com/users/show/10962834,Katharina,2014-01-02,https://a0.muscache.com/im/users/10962834/prof...,https://a0.muscache.com/im/users/10962834/prof...,NaN,1,158,NaN


In [254]:
hosts.to_csv('hosts.csv', encoding = 'utf-8')

In [250]:
hosts['host_response_rate'].astype('float64');